## Finding information on the file before reading it in

### Number of lines(row counts)

In [1]:
!wc -l Dataset/earthquakes.csv

9333 Dataset/earthquakes.csv


In [2]:
!wc -l Dataset/item_categories.csv

85 Dataset/item_categories.csv


In [3]:
!ls -lh Dataset | grep earthquakes.csv

-rw-rw-r-- 1 chinmayee chinmayee 3.4M May 27 13:57 earthquakes.csv


In [4]:
!awk -F',' '{print NF; exit}' Dataset/earthquakes.csv


26


In [5]:
!awk -F',' '{print NF; exit}' Dataset/item_categories.csv

2


In [6]:
import pandas as pd
df = pd.read_csv('Dataset/earthquakes.csv')

## Writing a DataFrame Object to a CSV File


#### Note that the index of df is just row numbers, so we don't want to keep it. Therefore, we pass index=False to the to_csv() method:

In [7]:
df.to_csv('output.csv', index=False)

## Writing a DataFrame Object to a Database

In [8]:
import sqlite3

In [9]:
with sqlite3.connect('Dataset/quakes.db') as connection:
    pd.read_csv('Dataset/tsunamis.csv').to_sql(
        'tsunamis', connection, index=False, if_exists='replace'
    )


### Making Pandas DataFrames from API Requests

### Get data from API

In [11]:
import datetime as dt
import pandas as pd
import requests

yesterday = dt.date.today() - dt.timedelta(days=1)
api = 'https://earthquake.usgs.gov/fdsnws/event/1/query'
payload = {
    'format':'geojson',
    'starttime': yesterday - dt.timedelta(days=30),
    'endtime': yesterday
    
}
response = requests.get(api, params=payload)

response.status_code

200

#### Response of 200 means OK, so we can pull the data out of the result. Since we asked the API for a JSON payload, we can extract it from the response with the json() method.

### Isolate the Data from the JSON Response


In [12]:
earthquake_json = response.json()
earthquake_json.keys()

dict_keys(['type', 'metadata', 'features', 'bbox'])

In [13]:
earthquake_json['metadata']

{'generated': 1653672464000,
 'url': 'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2022-04-26&endtime=2022-05-26',
 'title': 'USGS Earthquakes',
 'status': 200,
 'api': '1.13.6',
 'count': 9033}

In [14]:
type(earthquake_json['features'])

list

In [15]:
earthquake_json['features'][1]

{'type': 'Feature',
 'properties': {'mag': 2.6,
  'place': '56 km S of Whites City, New Mexico',
  'time': 1653522766400,
  'updated': 1653589914147,
  'tz': None,
  'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/tx2022kexs',
  'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=tx2022kexs&format=geojson',
  'felt': None,
  'cdi': None,
  'mmi': None,
  'alert': None,
  'status': 'reviewed',
  'tsunami': 0,
  'sig': 104,
  'net': 'tx',
  'code': '2022kexs',
  'ids': ',us7000hcjb,tx2022kexs,',
  'sources': ',us,tx,',
  'types': ',origin,phase-data,',
  'nst': 9,
  'dmin': 0.116324702,
  'rms': 0.2,
  'gap': 68,
  'magType': 'ml',
  'type': 'earthquake',
  'title': 'M 2.6 - 56 km S of Whites City, New Mexico'},
 'geometry': {'type': 'Point',
  'coordinates': [-104.3458278, 31.66428335, 7.905419922]},
 'id': 'tx2022kexs'}

### Convert to Dataframe


In [17]:
earthquake_properties_data = [quake['properties'] for quake in earthquake_json['features']]

df = pd.DataFrame(earthquake_properties_data)
df.head()

,mag,place,time,updated,tz,url,detail,felt,cdi,mmi,...,ids,sources,types,nst,dmin,rms,gap,magType,type,title
0,4.30,"231 km ENE of Lospalos, Timor Leste",1653523007692,1653525487040,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",us7000hcjf,",",us,",",origin,phase-data,",NaN,2.575000,0.47,65.0,mb,earthquake,"M 4.3 - 231 km ENE of Lospalos, Timor Leste"
1,2.60,"56 km S of Whites City, New Mexico",1653522766400,1653589914147,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",us7000hcjb,tx2022kexs,",",us,tx,",",origin,phase-data,",9.0,0.116325,0.20,68.0,ml,earthquake,"M 2.6 - 56 km S of Whites City, New Mexico"
2,5.00,"98 km NNW of Neiafu, Tonga",1653522062414,1653523671040,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",us7000hcja,",",us,",",origin,phase-data,",NaN,4.708000,0.63,125.0,mb,earthquake,"M 5.0 - 98 km NNW of Neiafu, Tonga"
3,0.36,"6km WNW of Cobb, CA",1653521910980,1653522913211,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",nc73737926,",",nc,",",nearby-cities,origin,phase-data,scitech-link,",7.0,0.001924,0.01,78.0,md,earthquake,"M 0.4 - 6km WNW of Cobb, CA"
4,1.17,"13km SE of Anza, CA",1653521862800,1653585747540,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",ci40026815,",",ci,",",focal-mechanism,nearby-cities,origin,phase-da...",44.0,0.031700,0.13,44.0,ml,earthquake,"M 1.2 - 13km SE of Anza, CA"


In [18]:
# Write to CSV
df.to_csv('earthquale.csv', index=False)